In [1]:
!pwd

/home/rj2357


In [1]:
%cd /scratch/rj2357/dls

/scratch/rj2357/dls


/share/apps/pyenv/py3.9/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
!pwd

/scratch/rj2357/dls


In [8]:
!pip install 'accelerate>=0.26.0'

Defaulting to user installation because normal site-packages is not writeable
  Using cached accelerate-1.2.1-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.2.1-py3-none-any.whl (336 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [6]:
!pip install peft

Defaulting to user installation because normal site-packages is not writeable
  Using cached peft-0.14.0-py3-none-any.whl.metadata (13 kB)
Using cached peft-0.14.0-py3-none-any.whl (374 kB)


In [2]:
!ls

 30911_dls.err	 __pycache__			   run.sh
 30911_dls.out	 config.py			   student1_model
'=0.15.0'	 datasets			   student1_trainer.py
'=0.16.0'	 eval.py			   student2_model
'=0.4.0'	 gemma-math-reasoning-lora	   test.jsonl
'=1.13.0'	 gemma-math-reasoning-lora-final   trainer.py
'=1.21.0'	 logs				   wandb
'=4.30.0'	 results


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

In [8]:
BASE_MODEL_DIR = "google/gemma-2b"  # Base model name from HuggingFace
LORA_ADAPTER_DIR = "models/gemma-math-reasoning-lora-final"

print("Loading the base model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_DIR, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_DIR,
    trust_remote_code=True,
    device_map="auto"
)

# Load LoRA adapter and merge it into the base model
print("Loading LoRA adapter...")
model = PeftModel.from_pretrained(base_model, LORA_ADAPTER_DIR)
model = model.merge_and_unload()  # Merge LoRA weights into the base model

# Set to evaluation mode
model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

Loading the base model and tokenizer...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading LoRA adapter...


GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-

In [9]:
def generate_prediction(model, tokenizer, question, max_length=512, temperature=0.7):
    """
    Generates a step-by-step reasoning and answer for a given math question.
    """
    input_text = f"Question: {question}\nLet's solve this step by step:"
    
    # Tokenize input and move tensors to the appropriate device
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    
    # Generate output
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode the output and return
    reasoning_and_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return reasoning_and_answer

In [13]:
question = "What is 15 multiplied by 4?"
response = generate_prediction(model, tokenizer, question)
print("Generated Response:")
print(response)

Generated Response:
Question: What is 15 multiplied by 4?
Let's solve this step by step:Certainly! Let's solve this problem step by step, showing each calculation clearly and explaining the reasoning.

### Step 1: Understand the Multiplication Operation
The multiplication operation is the same as adding the numbers together multiple times. In this case, we need to add 15 and 4 five times.

### Step 2: Perform the Multiplication
Let's break down the multiplication step by step:

1. **Multiply 15 by 4:**
   \[
   15 \times 4 = 15 + 15 + 15 + 15 + 15
   \]

2. **Add these multiplications:**
   - First, add 15 and 15:
     \[
     15 + 15 = 30
     \]
   - Next, add the result to 30:
     \[
     30 + 15 = 45
     \]
   - Add this to 45:
     \[
     45 + 15 = 60
     \]
   - Finally, add the result to 60:
     \[
     60 + 15 = 75
     \]

### Step 3: State the Final Answer
The result of 15 multiplied by 4 is 75.

### Summary
- We multiplied 15 by 4 by adding 15 + 15 + 15 + 15 + 15 repeat

In [11]:
question = "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May??"
response = generate_prediction(model, tokenizer, question)
print("Generated Response:")
print(response)

Generated Response:
Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May??
Let's solve this step by step:To solve this problem using algebra and mathematical notation, we can follow these steps:

1. **Define the variables:**
   - Let \( A \) represent the number of clips Natalia sold in April.
   - Let \( M \) represent the number of clips Natalia sold in May.

2. **Set up the equations based on the given information:**
   - Natalia sold clips to 48 of her friends in April:
     \[
     A = 48
     \]
   - In May, she sold half as many clips as she did in April:
     \[
     M = \frac{A}{2}
     \]

3. **Substitute the value of \( A \) into the equation for \( M \):**
   - Since \( A = 48 \):
     \[
     M = \frac{48}{2} = 24
     \]

4. **Calculate the total number of clips sold in April and May:**
   - The total number of clips sold is the sum of the clips sold in April an